In [1]:
import pickle
import graphlab as gl
import pandas as pd
from datetime import datetime, timedelta

from sklearn import cross_validation

[WARNING] graphlab.product_key: Unable to write current GraphLab Create license to /home/nawel/.graphlab/config. Ensure that this user account                         has write permission to /home/nawel/.graphlab/config to save the license for offline use.


This non-commercial license of GraphLab Create for academic use is assigned to nawel.medjkoune@u-psud.fr and will expire on December 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1483616300.log
[WARNING] graphlab.deploy._session: Unable to create session in specified location: '/home/nawel/.graphlab/artifacts'. Using: '/var/tmp/graphlab-nawel/12588/tmp_session_a0b7db58-71c7-4a67-a7ca-8bef716ff58d'
/home/nawel/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pickle.load(open("gpCalles.obj","rb"))

In [3]:
dataTest=pickle.load(open("testCalls.obj","rb"))

In [ ]:
print data.head()

In [5]:
# coding: utf-8
# Author:  Daro HENG <daro.heng@u-psud.fr>, 
# Licence: BSD 3 clause


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor                           
from sklearn.decomposition import PCA                                            
from sklearn.pipeline import Pipeline                                            
from sklearn.base import BaseEstimator    
from sklearn.neural_network import MLPRegressor
import numpy as np                                                               
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor


class Regressor(BaseEstimator):                                                  
    def __init__(self):    
        self.n_components = 8                                                 
        self.n_estimators = 200                                             
        self.learning_rate = 0.5                                                
        self.reg = Pipeline([                                      
               #('pca', PCA(n_components=self.n_components)),        
                            
                ###### kernel PCA
                #('pca',  KernelPCA(n_components=21,kernel="rbf",remove_zero_eig=False)),


                ###### Gradient Boosting Regressor
                ('reg', GradientBoostingRegressor(max_depth=10, min_samples_leaf=10,n_estimators=self.n_estimators,learning_rate=self.learning_rate,random_state=42))

                
                ###### SVR
                #('reg',SVR(C=1.0, cache_size=200, coef0=0.0, degree=6, epsilon=0.2, gamma='auto',kernel='rbf', max_iter=100, shrinking=True, tol=0.001, verbose=False))
                
                ##### linear model
                #('reg',linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=5, normalize=False))
               
                ##### MLP Regressor
                #('reg',MLPRegressor(hidden_layer_sizes=(7,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto',
                #learning_rate='constant', learning_rate_init=1, power_t=0.5, max_iter=1000, shuffle=True, random_state=42, 
                #tol=0.0001, verbose=False, warm_start=False, momentum=0.09, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1,
                #beta_1=0.9, beta_2=0.999, epsilon=1e-08))
                
               
                ##### Neighbors Regressor
                #('reg',RadiusNeighborsRegressor(radius=5.0))
                
                ##### PLS Regression
                #('reg', PLSRegression(copy=True, max_iter=500, n_components=2, scale=True,tol=1e-06))
                    
                ##### Decision Tree Regressor
                #('reg',DecisionTreeRegressor(max_depth=5))
                    
                ##### Random Forest Regressor
                #('reg',RandomForestRegressor(n_estimators=100))
                
                ##### Ada Boost Regressor avec Decision Tree Regressor
                #('reg', AdaBoostRegressor(DecisionTreeRegressor(max_depth=2),
                #        n_estimators=3000, random_state=42))
            ])                                                                   
                                                                                 
    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)


In [6]:
print "Start"
print "Loading the data train ..."

#print "Creat Mean_calls"
#soustraire la moyenne
#mean_calls=data.groupby(['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute']).mean().reset_index()
#data['MEAN_calls'] = pd.merge(data, mean_calls, how='left', on=['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute'])['CSPL_RECEIVED_CALLS_y']
#data['CSPL_RECEIVED_CALLS']=data['CSPL_RECEIVED_CALLS']-data['MEAN_calls']
#dataTest['MEAN_calls'] = pd.merge(dataTest, mean_calls, how='left', on=['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute'])['MEAN_calls']



print "Loading the X and Y train ..."
set_X_train=[]
set_Y_train=[]
i=0
while i < len(data['cod_ASS_ASSIGNMENT'].unique()):
	set_X_train.append(data[['DATE','DAY_WE_DS', 'year','month','day','hour','minute', 'cod_ASS_ASSIGNMENT']][data['cod_ASS_ASSIGNMENT']==(i)])
	set_Y_train.append(data['CSPL_RECEIVED_CALLS'][data['cod_ASS_ASSIGNMENT' ]==(i)])
	i=i+1

print "Loading the X  test ..."
set_X_test=[]
i=0
while i < len(data['cod_ASS_ASSIGNMENT'].unique()):
	set_X_test.append(dataTest[[ 'DATE','DAY_WE_DS', 'year','month','day','hour','minute', 'cod_ASS_ASSIGNMENT']][dataTest['cod_ASS_ASSIGNMENT' ]==(i)])
	i=i+1

i=0
listPred=[]
score_cv_global=[]
while i<len(set_X_train):
        from sklearn import preprocessing as pre
        scaler = pre.StandardScaler().fit(set_X_train[i][['DAY_WE_DS', 'year' ,'month','day','hour','minute']])
        X_train_scaled = scaler.transform(set_X_train[i][['DAY_WE_DS', 'year','month','day','hour','minute']])
        print " Train et Predict the categorie : ",i
        reg=Regressor()
        reg.fit(X_train_scaled, set_Y_train[i])
        #### Cross validation
        print "Cross validation ...", i
        #loo = cross_validation.LeaveOneOut(len(y_df))
        loo=10
        scores = cross_validation.cross_val_score(reg, X_train_scaled, set_Y_train[i], scoring='neg_mean_squared_error', cv=loo,)
        print "The score mean of cross validation : ", scores.mean()
        score_cv_global.append(scores.mean())

        if(len(set_X_test[i])>0):            
            X_test_scaled = scaler.transform(set_X_test[i][['DAY_WE_DS', 'year','month','day','hour','minute']])
            listPred.append( reg.predict(X_test_scaled))
        i=i+1


l=0
i=0
while l<len(set_X_test):
	if(len(set_X_test[l])>0):
		set_X_test[l]['CSPL_RECEIVED_CALLS'] =   listPred[i]
		i=i+1
	l=l+1



#on réassemble les valeurs de prédiction
resultPred= pd.concat(set_X_test)
resultPred=resultPred.sort_index()

def make_submission(test, prediction, filename='submission_.txt'):
	"""
	Create a submission file, 
	test: test dataset
	prediction: predicted values
	"""
	with open(filename, 'w') as f:
		f.write('DATE\tASS_ASSIGNMENT\tprediction\n')
		submission_strings = test['DATE'] + '\t' + test['ASS_ASSIGNMENT'] + '\t'+ prediction['CSPL_RECEIVED_CALLS'].astype(str)
		for row in submission_strings:
			f.write(row + '\n') 


#### write the submission
print "Write the submission ..."
make_submission(dataTest,resultPred)

print "End."
print "CV score global" , np.mean(score_cv_global)

Start
Loading the data train ...
Loading the X and Y train ...
Loading the X  test ...
 Train et Predict the categorie :  0
Cross validation ... 0
The score mean of cross validation :  -0.456131767998
 Train et Predict the categorie :  1
Cross validation ... 1
The score mean of cross validation :  -21.980871621
 Train et Predict the categorie :  2
Cross validation ... 2
The score mean of cross validation :  -0.00472808634775
 Train et Predict the categorie :  3
Cross validation ... 3
The score mean of cross validation :  -12.6734531725
 Train et Predict the categorie :  4
Cross validation ... 4
The score mean of cross validation :  -0.225255584868
 Train et Predict the categorie :  5
Cross validation ... 5
The score mean of cross validation :  -1.00006431634
 Train et Predict the categorie :  6
Cross validation ... 6
The score mean of cross validation :  -0.436753040048
 Train et Predict the categorie :  7
Cross validation ... 7
The score mean of cross validation :  -0.0080514946126
 T

In [19]:
resultPred['CSPL_RECEIVED_CALLS']=resultPred['CSPL_RECEIVED_CALLS'].apply(lambda x: x*(x>0))
resultPred['CSPL_RECEIVED_CALLS']=resultPred['CSPL_RECEIVED_CALLS'].apply(lambda x: math.ceil(x))

In [20]:
make_submission(dataTest,resultPred)

In [18]:
import math 
x=5455.2568
print math.ceil(x)

5456.0


In [14]:
from sklearn.metrics import  make_scorer

def linex(y, y_pred):
    import numpy as np
    return np.exp(-0.1*(y-y_pred))+0.1*(y-y_pred)-1
linex_scorer = make_scorer(linex, greater_is_better=False)
linex_scorer

make_scorer(linex, greater_is_better=False)

0

# time series (brouillon)

In [ ]:
df = data[['CSPL_RECEIVED_CALLS','DAY_WE_DS','cod_ASS_ASSIGNMENT','year','month','day','hour','minute', 'DATE']]
df=df[df.cod_ASS_ASSIGNMENT==1]
df.tail()

In [ ]:
import matplotlib.pyplot as plt
df.plot(x="DATE", y="CSPL_RECEIVED_CALLS", figsize=(14,4))
plt.show()

In [ ]:
from statsmodels.tsa.tsatools import detrend
notrend = detrend(df.CSPL_RECEIVED_CALLS)
df["notrend"] = notrend
df.plot(x="DATE", y=["CSPL_RECEIVED_CALLS", "notrend"], figsize=(14,4))

In [ ]:
plt.show()

In [ ]:
from statsmodels.tsa.stattools import acf
cor = acf(df.notrend)

In [ ]:
plt.plot(cor)
plt.show()

In [ ]:
from statsmodels.tsa.stattools import pacf
pcor = pacf(df.notrend)
plt.plot(pcor)
plt.show()

In [ ]:
df["diff"] = numpy.nan
df.ix[1, "diff"]=0
df.ix[2:, "diff"] = (df.iloc[1:, 0].as_matrix() - df.iloc[:len(df)-1, 0].as_matrix())
#pd.concat([df.head(n=5), df.tail(n=6)])
df.plot(x="DATE", y="diff", figsize=(14,4))
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARMA
arma_mod = ARMA(df["diff"].dropna().as_matrix(), order=(8, 1))
res = arma_mod.fit()
res.params



In [ ]:
from statsmodels.tsa.tsatools import lagmat
lag = 2
X = lagmat(df["diff"], lag)
lagged = df.copy()
for c in range(1,lag+1):
    lagged["lag%d" % c] = X[:, c-1]
#pd.concat([lagged.head(), lagged.tail()])

In [ ]:
xc = ["lag%d" % i for i in range(1,lag+1)]
split = 0.66
isplit = int(len(lagged) * split)
xt = lagged[10:][xc]
yt = lagged[10:]["diff"]
X_train, y_train, X_test, y_test = xt[:isplit], yt[:isplit], xt[isplit:], yt[isplit:]

In [ ]:
from sklearn.linear_model import LinearRegression
clr = LinearRegression()
clr.fit(X_train, y_train)

from sklearn.metrics import r2_score
r2 = r2_score(y_test.as_matrix(), clr.predict(X_test))
r2


In [ ]:
from sklearn.ensemble import RandomForestRegressor
clrf = RandomForestRegressor()
clrf.fit(X_train, y_train)



In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test.as_matrix(), clrf.predict(X_test))
r2

